## RSA Testbench


This notebook is to test the implementation of a RSA decryption running on the programmable logic. 

In [1]:
from pynq import Overlay
from pynq import MMIO
import numpy as np
import struct
import binascii
import math
import cmath
import random
import sympy
import matplotlib.pyplot as plt

NUM_SAMPLES = 10

In [2]:
ol=Overlay('./rsa.bit') #Change name of bitstream as required

In [3]:
rsa_ip=MMIO(0x40000000,10000) #Change base address as required

In [11]:
d_input=np.zeros(NUM_SAMPLES)
N_input=np.zeros(NUM_SAMPLES)
y_input=np.zeros(NUM_SAMPLES)
x_output=np.zeros(NUM_SAMPLES)

In [38]:
half_bit = 8
file_length = 100

def gen_p_q(bit):
    primes = [i for i in range(1 << (bit-2), min((1 << (bit - 2)) + 20000, (1 << (bit-1)))) if sympy.isprime(i)]
    #print(primes)
    while(True):
        p = random.choice(primes)
        if(sympy.isprime(2*p+1)):
            break
        
    while(True):
        q = random.choice(primes)
        if(sympy.isprime(2*q+1) and q != p):
            break
        
    return 2*p+1, 2*q+1


def gen_d(e, p, q):
    d = pow(e, -1, (p-1) * (q-1))
    return d

p, q = gen_p_q(half_bit)
e = 65537
d = gen_d(e, p, q)
n = p * q

In [39]:
# Helper function to split a large integer into 32-bit chunks
def pack_32bit_chunks(value):
    chunks = []
    while value > 0:
        chunks.append(value & 0xFFFFFFFF)  # Extract the least significant 32 bits
        value >>= 32  # Shift the value right by 32 bits
    if not chunks:  # If value is 0, we still need one 32-bit chunk
        chunks.append(0)
    return chunks


for i in range(NUM_SAMPLES):
    #Computing golden output
    gold_x = random.randint(1, (p-1) * (q-1) - 1)
    
    #Generating random inputs
    y=pow(gold_x, e, n)
    x2 = pow(y, d, n)
    print(y)
    print(d)
    print(n)
    print(x2)
    
    #Converting input to bytes to be sent to FPGA
    y_packed = struct.pack('<I', y)
    d_packed = struct.pack('<I', d)
    n_packed = struct.pack('<I', n)
    
    #Writing values to the FPGA
    rsa_ip.write(0x10,d_packed)                             #Change the offset as mentioned in vivado file
    rsa_ip.write(0x18,n_packed)                             #Change the offset as mentioned in vivado file
    rsa_ip.write(0x20,y_packed)
    
    #Starting and stopping the IP (Don't change this)
    rsa_ip.write(0x00,1)
    rsa_ip.write(0x00,0)
    
    #Reading from IP
    x=rsa_ip.read(0x28)                        #Change the offset as mentioned in vivado file
    
    # Displaying the result
    print(f"Result from FPGA: {x}")
    print(f"Golden x: {gold_x}")
    print(f"Type of x: {type(x)}")
    
    # Comparing with golden output
    # x (decrypted from FPGA) should match the original gold_x
    if x == gold_x:
        print(f"Test {i + 1}: PASS - FPGA output matches golden output")
    else:
        print(f"Test {i + 1}: FAIL - FPGA output does not match golden output")

18516
16937
29893
12409
Result from FPGA: 12409
Golden x: 12409
Type of x: <class 'int'>
Test 1: PASS - FPGA output matches golden output
29505
16937
29893
16840
Result from FPGA: 16840
Golden x: 16840
Type of x: <class 'int'>
Test 2: PASS - FPGA output matches golden output
19969
16937
29893
26792
Result from FPGA: 26792
Golden x: 26792
Type of x: <class 'int'>
Test 3: PASS - FPGA output matches golden output
24209
16937
29893
2411
Result from FPGA: 2411
Golden x: 2411
Type of x: <class 'int'>
Test 4: PASS - FPGA output matches golden output
18810
16937
29893
9205
Result from FPGA: 9205
Golden x: 9205
Type of x: <class 'int'>
Test 5: PASS - FPGA output matches golden output
13182
16937
29893
14800
Result from FPGA: 14800
Golden x: 14800
Type of x: <class 'int'>
Test 6: PASS - FPGA output matches golden output
21565
16937
29893
15209
Result from FPGA: 15209
Golden x: 15209
Type of x: <class 'int'>
Test 7: PASS - FPGA output matches golden output
17144
16937
29893
6772
Result from FPGA: